# Pré-Processamento dos Dados

### Importando as bibliotecas

In [1]:
import re
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import hstack, save_npz
from sklearn.model_selection import train_test_split
import os


### Carregando os dados

In [2]:
train = pd.read_csv("../data/raw/train.csv")
final_test = pd.read_csv("../data/raw/test.csv")

### Limpeza do texto

**Limpeza de Texto**: As estratégias foram cuidadosamente escolhidas com base na análise exploratória. Stopwords não foram removidas pois marcam hábitos de escrita distintos entre classes. Stemming e lemmatização foram evitados para preservar vícios de linguagem típicos de fake news.

**Transformações Aplicadas:**
- Separar pontuações: `!!!` → `! ! !` (enfatiza repetição)
- Substituir números: `2024` → `NUM` (generaliza quantidades)
- Marcar maiúsculas enfáticas: `IMPORTANTE` → `importante_ALLCAPS` (captura ênfase)

In [3]:
nltk.download("punkt")
nltk.download("punkt_tab")

def mark_caps(word):
    if word.isupper() and len(word) > 2:
        return word.lower() + "_ALLCAPS"
    return word

def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", " NUM ", text)
    text = re.sub(r"([!?.,])", r" \1 ", text)
    tokens = word_tokenize(text)
    tokens = [mark_caps(token) for token in tokens]
    return " ".join(tokens)

[nltk_data] Downloading package punkt to /home/vlsm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/vlsm/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [4]:
train["clean_text"] = train["text"].apply(clean_text)
train["clean_title"] = train["title"].apply(clean_text)

final_test["clean_text"] = final_test["text"].apply(clean_text)
final_test["clean_title"] = final_test["title"].apply(clean_text)

**Remoção de Colunas**: Excluindo `subject`, `date` e colunas originais de texto para evitar ruído no treinamento. O modelo focará apenas no texto e título processados.

In [5]:
train.drop(columns=["text", "title", "subject", "date"], inplace=True)
final_test.drop(columns=["text", "title", "subject", "date"], inplace=True)

In [6]:
train.head()

,id,label,clean_text,clean_title
0,13355,0,washington ( reuters ) - the u . s . departmen...,"exclusive : pentagon , lockheed near deal on $..."
1,2113,1,hill larious ! if this was a donald trump pi a...,“ hill ” arious…must see ! iowa parade goers t...
2,27667,0,manila ( reuters ) - philippine president rodr...,philippine leader says 'no way ' he 'll do dea...
3,15368,0,washington ( reuters ) - vice president joe bi...,biden asks u . s . congress to allow unencumbe...
4,6934,1,donald trump kicked off his monday morning by ...,trump claims ‘ any negative polls ’ are ‘ fake...


In [7]:
final_test.head()

,id,clean_text,clean_title
0,5398,after num_ALLCAPS months of hard-core islam mu...,obama ’ s “ clock boy ” comes back to texas…af...
1,5503,a group of volunteer soldiers announced this w...,lgbt volunteers aren ’ t waiting to be thrown ...
2,23151,bogota ( reuters ) - colombia s armed forces h...,colombia authorizes air raids against dissiden...
3,12669,( reuters ) - u . s . president donald trump a...,timeline : trump questions then honors `` one ...
4,27864,lima ( reuters ) - three peruvian policemen we...,three policemen killed in peru in drug-traffic...


### Vetorização

**Vetorização com TF-IDF**: Utiliza-se dois vetorizadores complementares:

**1. Vetorizador de Palavras**: Captura palavras importantes e compostas (bigramas)
- Range: 1-2 palavras
- Mínimo: 5 ocorrências | Máximo: 90% dos documentos
- Objetivo: Identificar tópicos principais

**2. Vetorizador de Caracteres**: Captura padrões de escrita e erros ortográficos
- Range: 3-4 caracteres
- Limite: 100k features para eficiência
- Objetivo: Detectar hábitos e vícios de linguagem

As matrizes são concatenadas para criar uma representação rica e multidimensional dos textos.

In [8]:
vectorizer_word = TfidfVectorizer(
    analyzer="word",
    ngram_range=(1,2),
    min_df=5,
    max_df=0.9,
    sublinear_tf=True
)

vectorizer_char = TfidfVectorizer(
    analyzer="char",
    ngram_range=(3,4),
    min_df=5,
    max_features=100000,
    dtype=np.float32
)

**Combinação de Vetorizadores**: Os vetores de palavras e caracteres são concatenados para criar uma representação conjunta.

In [9]:
def vectorize(text):
    X_word = vectorizer_word.transform(text)
    X_char = vectorizer_char.transform(text)
    return hstack([X_word, X_char])

def join_text(df):
    return (df["clean_text"] + " " + df["clean_title"]).fillna("")

**Divisão do Dataset**: Para avaliação adequada do modelo:

- **Treino (72%)**: Usado apenas para aprender os padrões
- **Validação (18%)**: Ajusta hiperparâmetros durante o treinamento
- **Teste (10%)**: Avaliação final e independente

Esta estratégia previne vazamento de dados e garante generalização real do modelo.

In [10]:
X_train_set, X_test, y_train_set, y_test = train_test_split(
    train.drop(columns=["label"]),
    train["label"],
    test_size=0.1,
    random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_set,
    y_train_set,
    test_size=0.2,
    random_state=42
)

In [11]:
train_text = join_text(X_train)
val_text = join_text(X_val)
test_text = join_text(X_test)
final_test_text = join_text(final_test)

In [12]:
X_word_train = vectorizer_word.fit_transform(train_text)
X_char_train = vectorizer_char.fit_transform(train_text)

In [13]:
X_train = hstack([X_word_train, X_char_train])
X_val = vectorize(val_text)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

X_test = vectorize(test_text)
y_test = y_test.reset_index(drop=True)

X_final_test = vectorize(final_test_text)

**Salvamento de Matrizes**: Os dados vetorizados são salvos em `data/processed/` para uso no treinamento do modelo.

In [17]:
os.makedirs("../data/processed", exist_ok=True)

save_npz("../data/processed/X_train.npz", X_train)
save_npz("../data/processed/X_val.npz", X_val)
save_npz("../data/processed/X_test.npz", X_test)
y_train.to_csv("../data/processed/y_train.csv", index=False)
y_val.to_csv("../data/processed/y_val.csv", index=False)
y_test.to_csv("../data/processed/y_test.csv", index=False)
save_npz("../data/processed/X_final_test.npz", X_final_test)